In [6]:
import sys
sys.path.append("..")

torch.cuda.set_device(0)

import sys
sys.path.append('../')
from datasets.backdoor_dataset import *
import numpy as np
import torch
from datasets.trans import *
import argparse
import pilgram

test_transform_cifar10 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    ])
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    ])

class BadEncoderBackdoor(Dataset):

    def __init__(self, numpy_file, aug_transform=None, clean_transform=None):
        """
        Args:
            numpy_file (string): Path to the numpy file.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.input_array = np.load(numpy_file)
        self.data = self.input_array['x']
        self.targets = self.input_array['y']


        self.test_transform = clean_transform
        self.aug_transform = aug_transform


    def __getitem__(self,index):
        img = copy.deepcopy(self.data[index])

        ########################
        img_ = Image.fromarray(img)
        img_clean_aug = self.aug_transform(img_)

        ###########################
        ### for ins filter only ###

        image_pil = Image.fromarray(img)
        filtered_image_pil = pilgram.xpro2(image_pil)
        img_backdoor_ins =self.test_transform(filtered_image_pil)

        ###########################

        # img[:] =img * self.trigger_mask_list[0] + self.trigger_patch_list[0][:]
        # img_backdoor_patch =self.test_transform(Image.fromarray(img))



        return img_clean_aug, img_backdoor_ins

    def __len__(self):
        return self.data.shape[0]




In [10]:
import torch.nn.functional as F


import torchvision.models as models
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import f1_score, roc_auc_score

# 加载预训练的ResNet50模型并修改最后一层以适应二分类任务
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 二分类
model = model.cuda()  # 确保模型在CUDA设备上


from torch.nn.functional import softmax


def test_model_with_metrics(model, test_loader):
    model.eval()  # 设置模型为评估模式
    correct = 0
    total = 0
    test_loss = 0.0
    y_true = []
    y_pred = []
    y_score = []

    with torch.no_grad():  # 在测试过程中不计算梯度
        for img_clean_aug, img_backdoor_ins in tqdm(test_loader):
            img_clean_aug = img_clean_aug.cuda()
            img_backdoor_ins = img_backdoor_ins.cuda()

            # 准备输入数据和标签
            inputs = torch.cat([img_clean_aug, img_backdoor_ins], dim=0)
            labels = torch.cat([torch.zeros(img_clean_aug.size(0)), torch.ones(img_backdoor_ins.size(0))], dim=0).long()
            inputs, labels = inputs.cuda(), labels.cuda()

            outputs = model(inputs)


            _, predicted = torch.max(outputs, 1)
            probs = softmax(outputs, dim=1)[:, 1]  # 获取正类的预测概率

            # 更新统计信息
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            y_score.extend(probs.cpu().numpy())

    accuracy = 100 * correct / total
    f1 = f1_score(y_true, y_pred)
    auroc = roc_auc_score(y_true, y_score)

    print(f'准确率: {accuracy:.2f}%, F1得分: {f1:.4f}, AUROC: {auroc:.4f}')

In [16]:
test_data_backdoor = BadEncoderBackdoor(numpy_file='../data/cifar10/train.npz', aug_transform=train_transform,clean_transform=test_transform_cifar10)
test_data_bd_dl = DataLoader(test_data_backdoor, batch_size=512, shuffle=True, num_workers=2)
test_model_with_metrics(model, test_data_bd_dl)

100%|██████████| 98/98 [01:33<00:00,  1.05it/s]

准确率: 54.48%, F1得分: 0.4545, AUROC: 0.5768


In [17]:
test_data_backdoor = BadEncoderBackdoor(numpy_file='../data/stl10/train.npz', aug_transform=train_transform,clean_transform=test_transform_cifar10)
test_data_bd_dl = DataLoader(test_data_backdoor, batch_size=512, shuffle=True, num_workers=2)
test_model_with_metrics(model, test_data_bd_dl)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:11<00:00,  1.16s/it]

准确率: 56.57%, F1得分: 0.4886, AUROC: 0.6096


In [14]:
test_data_backdoor = BadEncoderBackdoor(numpy_file='../data/svhn/train.npz', aug_transform=train_transform,clean_transform=test_transform_cifar10)
test_data_bd_dl = DataLoader(test_data_backdoor, batch_size=512, shuffle=True, num_workers=2)
test_model_with_metrics(model, test_data_bd_dl)

100%|██████████| 144/144 [02:26<00:00,  1.02s/it]


准确率: 51.94%, F1得分: 0.3858, AUROC: 0.5586


In [18]:
test_data_backdoor = BadEncoderBackdoor(numpy_file='../data/gtsrb/train.npz', aug_transform=train_transform,clean_transform=test_transform_cifar10)
test_data_bd_dl = DataLoader(test_data_backdoor, batch_size=512, shuffle=True, num_workers=2)
test_model_with_metrics(model, test_data_bd_dl)

100%|██████████| 77/77 [01:15<00:00,  1.02it/s]

准确率: 59.22%, F1得分: 0.5441, AUROC: 0.6379
